#### Imports:

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import mplcyberpunk
import numpy as np
import scipy as sp
import pandas as pd
import glob
import json
import os
import pickle
import corner
import pathlib
import itertools
import batman
import json
import re

from astropy.io import fits, ascii
from astropy import constants as c
from astropy import units as u
from astropy.time import Time
from matplotlib.collections import LineCollection
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil import parser
from PyAstronomy.pyTiming import pyPeriod

import utils

## Wavelength Calibration

Some of the lamp spectra through the target and comparison star slits are usually completely shifted off of the reference lamp spectra, so `guess_lines.py` can't be used. Instead of using this, the lines can be manually identified from [NOAO](http://iraf.noao.edu/specatlas/henear/henear.html). After getting the first target done, the rest can be bootstrapped relatively quickly since their arc spectra should be similar.

To make things a little easier, the following routine will automatically record the pixel and wavelength coordinate of each line selected from the bottom panel. To select the pixel value under the mouse, press 'X' on the keyboard. To record the corresponding wavelength value, click on the annotated value in the top reference panel. Rinse and repeat.

### Select lines

In [ ]:
# Reference (NOAO), do this one first
# dirpath = "./useful_scripts"
# fpath_arc_ref = f"{dirpath}/noao_flux.fits"
# fpath_lines_ref = f"{dirpath}/noao_line_list.txt"

# Reference (target) arc spectra, use as ref after calibrating inital with NOAO
dirpath_ref = "data_reductions/HATP26b/ut190313_a15_25_noflat/arcs"
fpath_arc_ref = f"{dirpath_ref}/HATP26b_2_arc.fits"
name = fpath_arc_ref.split('/')[-1].split('_')[0]
#fpath_lines_ref = f"{dirpath_ref}/{name}_guesses.txt"
fpath_lines_ref = f"{dirpath_ref}/{name}_lines_chips.csv"

# Arc spectra to compare
dirpath = "data_reductions/HATP26b/ut190313_a15_25_noflat/arcs"
fpath_arc = f"{dirpath}/c05_4_arc.fits"
name = fpath_arc.split('/')[-1].split('_')[0]
#fpath_lines = f"{dirpath}/{name}_guesses.csv"
fpath_lines = f"{dirpath}/{name}_lines_chips.csv"

# Plot
wavs, pixels = utils.compare_arc_lines(
    fpath_arc_ref=fpath_arc_ref, 
    fpath_lines_ref=fpath_lines_ref,
    fpath_arc=fpath_arc,
    fpath_lines=fpath_lines,
    sharex=False, sharey=False,
)

### Write to file

In [ ]:
df_guesses = pd.read_csv(fpath_lines, escapechar='#')
# update if lines chosen
#if len(wavs) != 0 and len(pixels) != 0:
chips = [int(fpath_arc.split('/')[-1].split('_')[1])] * len(wavs)
df_chosen = pd.DataFrame({"Wav":wavs, "Pix":pixels, "Chip":chips})

df_guesses = pd.concat([df_guesses, df_chosen])

fname = fpath_lines.split('/')[-1]
df_guesses = df_guesses.sort_values(by=["Chip", "Wav"])
display(df_guesses)
print(f"Will save to: {dirpath}/{fname}")

save = input("Continue? (y/n): ")
if save.lower() == 'y': 
    df_guesses.to_csv(f"{dirpath}/{fname}", index=False, escapechar='#')
    print(f"Saved to: {dirpath}/{fname}")
else: 
    print("not saved") 

In [ ]:
a, b, c = np.loadtxt(
    "data_reductions/HATP26b/ut190313_a15_25_noflat/arcs/HATP26b_lines_chips.csv",
    delimiter=',',
    unpack=True,
)

In [ ]:
a

### View wavelength solution

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=FIG_LARGE)

dir_lines = "HATP26b/data_reductions/ut190313_a15_25_noflat/arcs"
#fpaths = glob.glob(f"{dir_lines}/*guesses*.txt")
fpaths = glob.glob(f"{dir_lines}/*lines_chips.csv")
x_name = "Pix"
y_name = "Wav"
for fpath in fpaths:
    df = pd.read_csv(fpath, escapechar="#")
    comp_name = fpath.split('/')[-1].split('_')[0].split("comp")[-1]
    utils.plot_pix_wav(ax, df, x_name, y_name, comp_name)

ax.legend(frameon=True)
ax.set_xlabel(x_name)
ax.set_ylabel(y_name)

fig.tight_layout()